In [9]:
import requests
import collections
import json
import pandas as pd
import numpy as np

import numpy as np
from sklearn.decomposition import PCA

In [10]:
#https://plot.ly/python/3d-network-graph/
#import igraph as ig

import plotly
import plotly.plotly as py
plotly.tools.set_credentials_file(username='seth127', api_key='d5NicScxqZ59MSbd6mkY')

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
print(__version__) # requires version >= 1.9.0
import plotly.graph_objs as go
init_notebook_mode(connected=True)

3.1.1


# download data

In [243]:
animalia_link = "http://tolweb.org/onlinecontributors/app?service=external&page=xml/TreeStructureService&node_id=2374"
anlink1 = "http://tolweb.org/onlinecontributors/app?service=external&page=xml/TreeStructureService&node_id=2374&optype=1"
primate_link = "http://tolweb.org/onlinecontributors/app?service=external&page=xml/TreeStructureService&node_id=15963"
limit = 100000

In [244]:
raw = []
# raw format
#with requests.get(animalia_link, stream=True) as r:
with requests.get(primate_link, stream=True) as r:
    i = 0
    for c in r.iter_lines():
        c = c.decode('utf-8')#.strip()
        i +=1
        raw.append(c)
        if i % 500 == 0:
            print(i, end=' ')
        if i > limit:
            break

500 1000 1500 2000 

In [245]:
#with open("data/animalia_raw.xml", 'w') as f:
with open("data/primate_raw.xml", 'w') as f:
    f.write('\n'.join(raw))

In [7]:
# TreeML format
an_xml = []
attrs = []
tags = []
branches = {}
with requests.get(anlink1, stream=True) as r:
    i = 0
    for c in r.iter_lines():
        c = c.decode('utf-8').strip()
        i +=1
        #print(c.decode('utf-8'))
        an_xml.append(c)
        # get tags
        tags.append(c[1:].split(' ')[0])
        # get attribute options
        if "<attribute" in c:
            attr = c.split('name="')[1]
            attr = attr.split('"')[0]
            attrs.append(attr)
        if i % 500 == 0:
            print(i, end=' ')
        if i > limit:
            break

500 1000 1500 2000 2500 3000 3500 4000 4500 5000 5500 6000 6500 7000 7500 8000 8500 9000 9500 10000 10500 11000 11500 12000 12500 13000 13500 14000 14500 15000 15500 16000 16500 17000 17500 18000 18500 19000 19500 20000 20500 21000 21500 22000 22500 23000 23500 24000 24500 25000 25500 26000 26500 27000 27500 28000 28500 29000 29500 30000 30500 31000 31500 32000 32500 33000 33500 34000 34500 35000 35500 36000 36500 37000 37500 38000 38500 39000 39500 40000 40500 41000 41500 42000 42500 43000 43500 44000 44500 45000 45500 46000 46500 47000 47500 48000 48500 49000 49500 50000 50500 51000 51500 52000 52500 53000 53500 54000 54500 55000 55500 56000 56500 57000 57500 58000 58500 59000 59500 60000 60500 61000 61500 62000 62500 63000 63500 64000 64500 65000 65500 66000 66500 67000 67500 68000 68500 69000 69500 70000 70500 71000 71500 72000 72500 73000 73500 74000 74500 75000 75500 76000 76500 77000 77500 78000 78500 79000 79500 80000 80500 81000 81500 82000 82500 83000 83500 84000 84500 85000 

In [8]:
print(set(attrs))
print(collections.Counter(tags))

{'description', 'id', 'name', 'isLeaf', 'image', 'contributors'}
Counter({'attribute': 73158, 'branch>': 22245, '/branch>': 4583, 'attributeDecl': 6, '': 1, 'tree>': 1, 'hite-masked': 1, 'nd': 1, 'ellow-browed': 1, 'ufous-crowned': 1, 'and-tailed': 1, 'declarations>': 1, '?xml': 1})


In [9]:
an_xml[10:30]

['<branch>',
 '<attribute name="name" value="Animals"/>',
 '<attribute name="id" value="2374"/>',
 '<attribute name="image" value="personglobe.png"/>',
 '<attribute name="description" value="(Metazoa)"/>',
 '<attribute name="contributors" value="Gonzalo Giribet"/>',
 '<branch>',
 '<attribute name="name" value=""/>',
 '<attribute name="id" value="2458"/>',
 '<branch>',
 '<attribute name="name" value="Bilateria"/>',
 '<attribute name="id" value="2459"/>',
 '<attribute name="image" value="globe.png"/>',
 '<attribute name="description" value="(most animals including vertebrates, arthropods, molluscs, etc.)"/>',
 '<branch>',
 '<attribute name="name" value="Deuterostomia"/>',
 '<attribute name="id" value="2466"/>',
 '<attribute name="image" value="globe.png"/>',
 '<attribute name="description" value="(vertebrates, echinoderms, tunicates, etc.)"/>',
 '<branch>']

In [10]:
with open("data/animalia_treeml.xml", 'w') as f:
    f.write('\n'.join(an_xml))

#### branches

In [11]:
def get_key(line):
    attr = line.split('name="')[1]
    attr = attr.split('"')[0]
    return attr

def get_value(line):
    try:
        attr = line.split('value="')[1]
        attr = attr.split('"')[0]
    except:
        attr = 'error'
    return attr

In [12]:
branches = {}
this_name = 'default'
branches[this_name] = {}
with requests.get(anlink1, stream=True) as r:
    i = 0
    #
    for c in r.iter_lines():
        c = c.decode('utf-8').strip()
        i +=1
        if "<attribute" in c:
            this_key = get_key(c)
            this_value = get_value(c)
            if this_key == 'name':
                if this_value == '':
                    this_name = "empty_{}".format(i)
                else:
                    this_name = this_value
                branches[this_name] = {}
            else:
                branches[this_name][this_key] = this_value
        #
        if i > 5000:
            break

In [13]:
len(branches.keys())

1273

In [14]:
branches['Sphenacodontoidea']

{'id': '14971'}

# Load data from file

In [539]:
#with open("data/animalia_raw.xml", 'r') as f:
with open("data/primate_raw.xml", 'r') as f:
    raw = f.read().split('\n')
print(len(raw))

#with open("data/animalia_treeml.xml", 'r') as f:
#    an_xml = f.read().split('\n')
#print(len(an_xml))

2472


In [540]:
# parsing...
little_raw = raw[:10000]
#nodes_raw = [n.strip() for n in little_raw if "ID=" in n]
nodes_raw = []
for i in range(len(little_raw)):
    n = little_raw[i]
    try:
        nplus = little_raw[i+1]
    except:
        nplus = "<no next>"
    if "ID=" in n:
        nodes_raw.append(n.strip() + nplus.strip())
print(len(nodes_raw))
n = nodes_raw[0]
print(n)

218
<NODE EXTINCT="0" ID="15963" CONFIDENCE="0" PHYLESIS="0" LEAF="0" HASPAGE="1" ANCESTORWITHPAGE="15997" ITALICIZENAME="0" INCOMPLETESUBGROUPS="0" SHOWAUTHORITY="0" SHOWAUTHORITYCONTAINING="0" IS_NEW_COMBINATION="0" COMBINATION_DATE="null" CHILDCOUNT="2"><NAME><![CDATA[Primates]]></NAME>


#### nodes

In [541]:
def split_stat(n, start, end):
    try:
        this_stat = n.split(start)[1].split(end)[0]
    except:
        this_stat = "no {}".format(start)
    return this_stat

In [542]:
nodes_list = []
for n in nodes_raw:
    # get grouping
    this_extinct = int(split_stat(n, 'NODE EXTINCT="', '"'))
    # get phylesis
    this_phylesis = int(split_stat(n, 'PHYLESIS="', '"'))
    # get id
    this_id = int(split_stat(n, 'ID="', '"'))
    # get name
    this_name = split_stat(n, 'CDATA[', ']')
    # get num ancestors
    this_ancestor = int(split_stat(n, 'ANCESTORWITHPAGE="', '"'))
    # get num kids
    this_num_kids = int(split_stat(n, 'CHILDCOUNT="', '"'))
    # assign
    nodes_list.append({
        'extinct' : this_extinct,
        'id' : this_id,
        'phylesis' : this_phylesis,
        'name' : this_name,
        'ancestor' : this_ancestor,
        'num_kids' : this_num_kids
    })
    

In [543]:
print(len(nodes_list))
nodes_list[0]

218


{'ancestor': 15997,
 'extinct': 0,
 'id': 15963,
 'name': 'Primates',
 'num_kids': 2,
 'phylesis': 0}

#### filter to only named creatures

In [544]:
nodes_list = [n for n in nodes_list if 'CDATA' not in n['name']]
print(len(nodes_list))

166


### create x y z

In [545]:
def rn():
    return np.random.uniform(-1,1,1)[0]

for node in nodes_list:
    node['depth'] = None
    node['x'] = None
    node['y'] = None
    
# assign first one to depth 0
nodes_list[0]['depth'] = 1
nodes_list[0]['x'] = rn()
nodes_list[0]['y'] = rn()



In [546]:
for i in range(len(nodes_list)):
    node = nodes_list[i]
    parent = [n for n in nodes_list if n['id'] == node['ancestor']]
    if len(parent) > 0:
        nodes_list[i]['depth'] = parent[0]['depth'] + 1
        # depth multiplier for narrowing
        this_depth = nodes_list[i]['depth']
        #if this_depth == 1:
        #    depth_mult = 0
        #else: 
        #    depth_mult = (3/this_depth)
        #depth_mult = (3/this_depth)
        depth_mult = 1
            
        nodes_list[i]['x'] = parent[0]['x'] + (rn() * depth_mult)
        nodes_list[i]['y'] = parent[0]['y'] + (rn() * depth_mult)


#### links

In [547]:
links_list = []
for n in nodes_list:
    links_list.append({
        'source': n['id'], 
        'target': n['ancestor'], 
        'value': n['num_kids']
    })

In [548]:
links_list_uno = []
for n in nodes_list:
    links_list_uno.append({
        'source': n['id'], 
        'target': n['ancestor'], 
        'value': 1
    })

In [549]:
links_list[:3]

[{'source': 15963, 'target': 15997, 'value': 2},
 {'source': 16292, 'target': 15963, 'value': 2},
 {'source': 16308, 'target': 16292, 'value': 0}]

### prep df

In [550]:
df = pd.DataFrame(nodes_list)
#df = df.set_index('id')
df.head()

,ancestor,depth,extinct,id,name,num_kids,phylesis,x,y
0,15997,1,0,15963,Primates,2,0,0.738548,0.860418
1,15963,2,0,16292,Platyrrhini,2,0,0.896368,1.210393
2,16292,3,0,16308,Callithrix,0,0,0.907100,2.062635
3,16292,3,0,16309,Cebuella,0,0,0.359640,1.785354
4,16292,3,0,16310,Leontopithecus,0,0,0.450032,1.579830


In [551]:
print("phylesis: {}".format(collections.Counter(df['phylesis'])))
print("extinct: {}".format(collections.Counter(df['extinct'])))

phylesis: Counter({0: 162, 1: 3, 2: 1})
extinct: Counter({0: 133, 2: 33})


In [552]:
max(df['depth'])

5

### manual igraph prep

In [553]:
max_nodes = 5000
max_depth = 10
color_attr = 'extinct'
Z_dim = 'depth'

In [554]:
# subset data for plotting
df = df[df['depth'] <= max_depth].head(max_nodes)
df = df.reset_index()
print(df.shape)

(166, 10)


#### filter

In [555]:
search_name = "Animal"
df[df['name'].str.contains(search_name)]

,index,ancestor,depth,extinct,id,name,num_kids,phylesis,x,y


In [556]:
#pick = 16293 # great apes
pick = 16418 # homo
#pick = 2374 # animals

In [557]:

ma = int(df['ancestor'][df['id'] == pick])
print(ma)
ancestors = []

this_id = pick
for i in range(max_depth):
    this_row = df[df['id'] == this_id]
    if this_row.shape[0] == 0:
        break
    else:
        this_ancestor = int(this_row['ancestor'])
        ancestors.append(this_ancestor)
        this_id = this_ancestor
    
print(ancestors)

16299
[16299, 16293, 15963, 15997]


In [558]:
kin = [pick]

# add descendants
for i, row in df.iterrows():
    if int(row['ancestor']) in kin:
        kin.append(int(row['id']))

# add siblings
for i, row in df.iterrows():
    if int(row['ancestor']) == ma:
        kin.append(int(row['id']))

# add ancestors
kin += ancestors
print(len(kin))

16


In [559]:
# subset
df['kin'] = 0
df['kin'][df['id'].isin(kin)] = 1
df.head()

/Users/Seth/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,index,ancestor,depth,extinct,id,name,num_kids,phylesis,x,y,kin
0,0,15997,1,0,15963,Primates,2,0,0.738548,0.860418,1
1,1,15963,2,0,16292,Platyrrhini,2,0,0.896368,1.210393,0
2,2,16292,3,0,16308,Callithrix,0,0,0.907100,2.062635,0
3,3,16292,3,0,16309,Cebuella,0,0,0.359640,1.785354,0
4,4,16292,3,0,16310,Leontopithecus,0,0,0.450032,1.579830,0


In [560]:
links_list = [l for l in links_list if l['target'] in list(df['id'])]
print(len(links_list))

165


In [561]:
# # PCA, not needed if not using PCA
# X = np.matrix(df[['group', 'num_kids', 'ancestor', 'phylesis']])
# pca = PCA(n_components=2)
# X_pca = PCA(n_components=2).fit(X).transform(X)

In [562]:
##data = {'links' : links_list[:max_nodes], 'nodes' : nodes_list[:max_nodes]}
#plot_nodes = [n for n in nodes_list if n['depth'] <= max_depth][:max_nodes]
#plot_node_ids = [n['id'] for n in plot_nodes]
#data = {'links' : [l for l in links_list if l['target'] in plot_node_ids], 
#        'nodes' : plot_nodes}

In [563]:
L=len(links_list)
print(L)
Edges=[(links_list[k]['source'], links_list[k]['target']) for k in range(L)]
print(Edges[:5])
#G=ig.Graph(Edges, directed=False)

165
[(16292, 15963), (16308, 16292), (16309, 16292), (16310, 16292), (16311, 16292)]


In [564]:
labels=[]
group=[]
alpha = []
layt = []
for i, node in df.iterrows():
    # create text labels
    labels.append(str(node['depth']) + ' ' + node['name'])
    # create color key
    group.append(node[color_attr])
    # create opacity key
    alpha.append(node['kin'])
    # create layout list
    d = node[Z_dim]
    # # PCA
    # # layt.append(list(X_pca[i]) + [d * -1])
    layt.append([node['x'], 
                 node['y'], 
                 (d*-1)+np.random.uniform(-0.1,0.1,1)[0]])    
    

print(labels[:3])
print(group[:3])
print(alpha[:3])
print(len(layt))
layt[:3]

['1 Primates', '2 Platyrrhini', '3 Callithrix']
[0, 0, 0]
[1, 0, 0]
166


[[0.7385482625055664, 0.8604176442244436, -1.0771024305350483],
 [0.8963676102132674, 1.2103925104285014, -2.0412750722409498],
 [0.9070999326919809, 2.0626345047866756, -3.0481671713614618]]

In [565]:
Xn=[layt[k][0] for k in range(len(layt))]# x-coordinates of nodes
Yn=[layt[k][1] for k in range(len(layt))]# y-coordinates
Zn=[layt[k][2] for k in range(len(layt))]# z-coordinates

# check if they're kinfolk
Xnk=[layt[k][0] for k in range(len(layt)) if alpha[k]==1]# x-coordinates of nodes
Ynk=[layt[k][1] for k in range(len(layt)) if alpha[k]==1]# y-coordinates
Znk=[layt[k][2] for k in range(len(layt)) if alpha[k]==1]# z-coordinates
groupk=[group[k] for k in range(len(layt)) if alpha[k]==1]# color
print(len(Xnk))

14


In [566]:
Xe=[]
Ye=[]
Ze=[]
Xek=[]
Yek=[]
Zek=[]
for e in Edges:
    try: 
        e0 = df[df['id'] == e[0]].index.values[0]
    except:
        e0 = 0
    try:
        e1 = df[df['id'] == e[1]].index.values[0]
    except:
        e1 = 0
    #
    Xe+=[layt[e0][0],layt[e1][0], None]# x-coordinates of edge ends
    Ye+=[layt[e0][1],layt[e1][1], None]  
    Ze+=[layt[e0][2],layt[e1][2], None] 
    # check if they're kinfolk
    if e[0] in kin:
        Xek+=[layt[e0][0],layt[e1][0], None]
        Yek+=[layt[e0][1],layt[e1][1], None]
        Zek+=[layt[e0][2],layt[e1][2], None]

In [567]:
print(len(Ze))
print(len(Zek))

495
39


In [568]:
df.loc[df['id']==pick]['name'].values[0]

'Homo'

## plotly

In [569]:
this_title = df.loc[0, 'name']
this_text = df.loc[df['id']==pick]['name'].values[0]

# lines
trace1=go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               opacity=0.7,
               line=dict(color='rgb(125,125,125)', width=1),
               hoverinfo='none'
               )

In [570]:
# kinfolk lines
trace1k=go.Scatter3d(x=Xek,
               y=Yek,
               z=Zek,
               mode='lines',
               opacity=1,
               line=dict(color='rgb(125,125,125)', width=1.5),
               hoverinfo='none'
               )

In [571]:
# nodes
trace2=go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='actors',
               marker=dict(symbol='circle',
                             size=6,
                             color=group,
                             opacity=0.6,
                             colorscale='Viridis',
                             line=dict(color='rgb(50,50,50)', width=0.5)
                             ),
               text=labels,
               hoverinfo='text'
               )

In [572]:
# kinfolk nodes
trace2k=go.Scatter3d(x=Xnk,
               y=Ynk,
               z=Znk,
               mode='markers',
               name='actors',
               marker=dict(symbol='circle',
                             size=8,
                             color=groupk,
                             colorscale='Viridis',
                             line=dict(color='rgb(50,50,50)', width=0.5)
                             ),
               text=labels,
               hoverinfo='text'
               )

In [573]:
axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

layout = go.Layout(
         title=this_title,
         width=1000,
         height=1000,
         showlegend=False,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
        ),
     margin=dict(
        t=100
    ),
    hovermode='closest',
    annotations=[
           dict(
           showarrow=False,
            text=this_text,
            xref='paper',
            yref='paper',
            x=0,
            y=0.1,
            xanchor='left',
            yanchor='bottom',
            font=dict(
            size=14
            )
            )
        ],    )

In [574]:
plot_data=[trace1, trace1k, trace2, trace2k]
#plot_data=[trace1]
fig=go.Figure(data=plot_data, layout=layout)

iplot(fig)

In [575]:
py.iplot(fig, filename = 'primates')
#py.iplot(fig, filename = 'animals')

In [388]:
#layt.savetext("layt_numpy.txt")

In [184]:
#layt.savetxt("layt_numpy.txt")